## 1. Import Library 

In [120]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from sklearn.metrics  import accuracy_score,f1_score,precision_score,recall_score
from keras.activations import relu
from keras.utils import to_categorical
import numpy as np 
import regex as re 
np.set_printoptions(threshold=np.inf)

## 2. Data Proprocesing

#### 2.1. Read the file and divide it into sentences

In [121]:
def file_to_sentence_list(file_path):
    with open(file_path, 'r') as file:
        text = file.read()
    sentences = [sentence.strip() for sentence in re.split(r'(?<=[.!?])\s+', text) if sentence.strip()]
    return sentences 

#### 2.2. Create a dictionary for each word in the text file

In [122]:
file_path = 'train.txt'
text_data = file_to_sentence_list(file_path)
print(text_data)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data) 
total_words = len(tokenizer.word_index) + 1
print(tokenizer.word_index)


['Ho Chi Minh City University of Technology and Education, with a long and prestigious history in the field of education and research, is a modern school with a rich and diverse learning and research environment.', 'Located in the city center, Ho Chi Minh City University of Technology and Education is not only a provider of professional knowledge but also a vibrant and creative academic community.', 'With modern facilities, classrooms, laboratories and libraries are fully equipped, creating favorable conditions for students and lecturers in the learning and research process.', 'The school is also proud of its team of experienced, highly qualified lecturers who are committed to providing students with the best support and development.', 'Training programs are diverse, flexible and reflect the actual needs of the labor market, giving students the opportunity to develop themselves and prepare for their future careers.', 'In addition, the school also promotes international research and coo

#### 2.3. Create Input 

In [123]:
def CreateInput(text_data):
    input_sequences = []
    for line in text_data:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences
input_sequences=CreateInput(text_data)
print(input_sequences)

[[20, 21], [20, 21, 22], [20, 21, 22, 12], [20, 21, 22, 12, 13], [20, 21, 22, 12, 13, 3], [20, 21, 22, 12, 13, 3, 23], [20, 21, 22, 12, 13, 3, 23, 1], [20, 21, 22, 12, 13, 3, 23, 1, 9], [20, 21, 22, 12, 13, 3, 23, 1, 9, 6], [20, 21, 22, 12, 13, 3, 23, 1, 9, 6, 7], [20, 21, 22, 12, 13, 3, 23, 1, 9, 6, 7, 29], [20, 21, 22, 12, 13, 3, 23, 1, 9, 6, 7, 29, 1], [20, 21, 22, 12, 13, 3, 23, 1, 9, 6, 7, 29, 1, 30], [20, 21, 22, 12, 13, 3, 23, 1, 9, 6, 7, 29, 1, 30, 31], [20, 21, 22, 12, 13, 3, 23, 1, 9, 6, 7, 29, 1, 30, 31, 4], [20, 21, 22, 12, 13, 3, 23, 1, 9, 6, 7, 29, 1, 30, 31, 4, 2], [20, 21, 22, 12, 13, 3, 23, 1, 9, 6, 7, 29, 1, 30, 31, 4, 2, 32], [20, 21, 22, 12, 13, 3, 23, 1, 9, 6, 7, 29, 1, 30, 31, 4, 2, 32, 3], [20, 21, 22, 12, 13, 3, 23, 1, 9, 6, 7, 29, 1, 30, 31, 4, 2, 32, 3, 9], [20, 21, 22, 12, 13, 3, 23, 1, 9, 6, 7, 29, 1, 30, 31, 4, 2, 32, 3, 9, 1], [20, 21, 22, 12, 13, 3, 23, 1, 9, 6, 7, 29, 1, 30, 31, 4, 2, 32, 3, 9, 1, 5], [20, 21, 22, 12, 13, 3, 23, 1, 9, 6, 7, 29, 1, 30, 31

#### 2.4.Convert to full vectors

In [124]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

#### 2.5 Divide the data set into training set and testing set

In [125]:
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y =to_categorical(y, num_classes=total_words)

### 3 RNN Model

#### 3.1 Configuration RNN

In [126]:
model = Sequential()
model.add(Embedding(total_words, 10, name='embedding_layer')) 
model.add(SimpleRNN(8, return_sequences=True, activation=relu, name='rnn_layer_1'))
model.add(SimpleRNN(9, return_sequences=True, activation=relu, name='rnn_layer_2'))
model.add(SimpleRNN(9, activation=relu, name='rnn_layer_3'))
model.add(Dense(total_words, activation='softmax', name='output_layer'))


#### 3.2. Trainning RNN

In [127]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=300, verbose=1)

Epoch 1/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - accuracy: 0.0117 - loss: 4.6258
Epoch 2/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.0141 - loss: 4.6221   
Epoch 3/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.0750 - loss: 4.6181
Epoch 4/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.0558 - loss: 4.6153
Epoch 5/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.1242 - loss: 4.6098
Epoch 6/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.0826 - loss: 4.6060
Epoch 7/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.1117 - loss: 4.6012
Epoch 8/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.0639 - loss: 4.5929
Epoch 9/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.1027 - loss: 4.5845
Epoch 10/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.0950 - loss: 4.5707
Epoch 11/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.0838 - loss: 4.5545
Epoch 12/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.0548 -

#### 3.3 Model Summary

In [128]:
model.summary()

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_layer (Embedding)     │ (None, 34, 10)         │         1,020 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rnn_layer_1 (SimpleRNN)         │ (None, 34, 8)          │           152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rnn_layer_2 (SimpleRNN)         │ (None, 34, 9)          │           162 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rnn_layer_3 (SimpleRNN)         │ (None, 9)              │           171 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, 102)            │         1,020 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,577 (29.60 KB)

 Trainable params: 2,525 (9.86 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 5,052 (19.74 KB)

#### 3.4 Predicting 

In [129]:
next_words = 10
seed_text="In short, the University of Technical Education is an ideal"
y_predict=[]
def Recommend(seed_text):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted_probs = model.predict(token_list)
        predicted_word = tokenizer.index_word[np.argmax(predicted_probs)]
        seed_text += " " + predicted_word
        y_predict.append(predicted_word)
    return seed_text
y_result=Recommend(seed_text)
print(y_result)
print(y_predict)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 706ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
In short, the University of Technical Education is an ideal learning and research environment where creativity and discovery are awakened
['learning', 'and', 'research', 'environment', 'where', 'creativity', 'and', 'discovery', 'are', 'awakened']


#### 3.5 Evaluating 

In [130]:
y_true = ["learning","and","research" ,"environment","where","creativity","and","discovery","are","awakened"]
accuracy = accuracy_score(y_true, y_predict)
print(f'Accuracy: {accuracy:.4f}')

precision = precision_score(y_true, y_predict, average='weighted')
print(f"Precision: {precision:.4f}")

recall = recall_score(y_true, y_predict, average='weighted')
print(f"Recall: {recall:.4f}")

f1score = f1_score(y_true, y_predict, average='weighted')
print(f"F1-score: {f1score}")

Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
F1-score: 1.0
